## DNN Models for Energy Forecasting

The code to use for DNN model of the energy forecast project

In [1]:
import tensorflow as tf
print(tf.__version__)

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.8.0


In [ ]:
tf.estimator.RunConfig

In [2]:
# change these to try this notebook out
BUCKET = 'atos-asl'
PROJECT = 'qwiklabs-gcp-aebfb78fe0f1b1d1'
REGION = 'us-central1'

In [3]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '1.8'  

In [4]:
%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [7]:
%%bash
OUTDIR=gs://${BUCKET}/trained_data/dnn/
JOBNAME=energy_forecasting_dnn_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
  --region=$REGION \
  --runtime-version=$TFVERSION \
  --scale-tier=STANDARD-1 \
  --staging-bucket=gs://$BUCKET \
  --module-name=atos-asl.task \
  --package-path=$(pwd)/atos-asl \
  --job-dir=$OUTDIR \
  -- \
  --eval_set=gs://${BUCKET}/datasets/avg_weather_eval_set.csv \
  --train_set=gs://${BUCKET}/datasets/avg_weather_train_set.csv \
  --output_dir=${OUTDIR}

gs://atos-asl/trained_data/dnn/ us-central1 energy_forecasting_dnn_180914_215621
jobId: energy_forecasting_dnn_180914_215621
state: QUEUED


CommandException: 1 files/objects could not be removed.
Job [energy_forecasting_dnn_180914_215621] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe energy_forecasting_dnn_180914_215621

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs energy_forecasting_dnn_180914_215621


In [12]:
from google.datalab.ml import TensorBoard
TensorBoard().start('gs://{}/trained_data/'.format(BUCKET))

TensorBoard was started successfully with pid 21168. Click here to access it.

21168

In [11]:
for pid in TensorBoard.list()['pid']:
  TensorBoard().stop(pid)
  print('Stopped TensorBoard with pid {}'.format(pid))

Stopped TensorBoard with pid 21138
